In [1]:
import sys
sys.path.append('/home/myk/min0')
sys.path.append('/home/myk/min0/traffic-classification')
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# custom Layer
import layer 
import DataRepresentation.Label2Int as pp
from layer import EmbeddingLayer
from layer import MultiHeadAttentionLayer
from layer import FeedForwardLayer


In [2]:
import random

def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

In [3]:
x = np.load('/home/myk/min0/x_tuple2token_900.npy')
y = pp.getlabel('/home/myk/min0/metadata.csv')
print(x.shape)
num_index = np.max(x)

0 is arkio
1 is bigballers
2 is bigscreen
3 is blaston
4 is campfire
5 is epicrollercoasters
6 is firsthand
7 is firsttouch
8 is fitxr
9 is godsofgravity
10 is gorillatag
11 is gunraiders
12 is horizonworkrooms
13 is horizonworlds
14 is hyperdash
15 is immersed
16 is netflix
17 is roblox
18 is vrchat
19 is youtube
(3180, 900)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device ="cpu"
print(device)

cuda


In [5]:
batch = 128
lr = 0.0001
epoch = 200

embedding_dim = 256
n_layer = 4
n_head = 4
dim_ff = 128
dropout = 0.3
output_size = 20

In [6]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# train, test dataset split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,shuffle=True, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = CustomDataset(x_train,y_train)
test_dataset = CustomDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True,drop_last=True)

print(x_train.shape)
print(x_test.shape)
print(batch)

(2544, 900)
(636, 900)
128


In [7]:
class classifier(nn.Module):
    def __init__(self, seq_len, output_size, dr_rate=0,device="cuda"):
        super(classifier, self).__init__()
        
        
        self.layer1 = nn.Sequential(
                nn.Conv1d(in_channels=seq_len, out_channels=seq_len*2, kernel_size=1, stride=1),
                nn.GELU(),
                nn.Dropout(dr_rate)
            ).to(device)
        self.layer2 = nn.Sequential(
                nn.Conv1d(in_channels=seq_len*2, out_channels=seq_len*3, kernel_size=1, stride=1),
                nn.GELU(),
                nn.Dropout(dr_rate)
            ).to(device)
            
        self.ff = nn.Sequential(nn.Linear(691200,2000),
                                nn.GELU(),
                                nn.Linear(2000,output_size)).to(device)
        
    def forward(self, x):
        batch = x.shape[0]
        out = x
        out = self.layer1(out)
        out = self.layer2(out)
        out = out.reshape(batch, -1)
        out = self.ff(out)
        out1 = out.to(device)
        return out1     
        
        

In [8]:
class transformer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim,  n_layer, n_head, dim_ff, dropout, output_size, device="cuda"):
        
        super(transformer, self).__init__()
        
        token_embed = EmbeddingLayer.TokenEmbedding(max_len = num_embeddings, d_embed=embedding_dim)
        pos_embed = EmbeddingLayer.PositionalEmbedding(d_embedding=embedding_dim, max_len = num_embeddings, device=device)
        self.embedding = EmbeddingLayer.Embedding(token_embed, pos_embed).to(device)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=n_head, dim_feedforward=dim_ff, dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=n_layer)
        
        self.classifier = classifier( 900 ,output_size,dropout).to(device)
        
    def forward(self, x, mask):
        out = self.embedding(x)
        #print("after embedding",out.shape)
        out = self.encoder(out, src_key_padding_mask = mask)
        out = self.classifier(out)
        return out

In [9]:
model = transformer(num_embeddings=num_index+1, embedding_dim= embedding_dim, n_layer=n_layer, n_head=n_head, dim_ff=dim_ff, dropout=dropout,output_size=output_size, device=device )

In [10]:
model = torch.nn.DataParallel(model).to(device)

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optim,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

In [12]:
def src_key_padding_mask(x, pad_idx = 0):
    '''
    input : x [ batch, seq_len ]
    output : mask [batch, seq_len]
    input애서 pad_idx인 부분을 true로, 나머지는 false로 채운 mask를 생성하는 함수 
    '''
    mask = (x == pad_idx)
    return mask

In [13]:

for i in range(epoch):
    for j,[data,label] in enumerate(train_loader):
        label = label.type(torch.LongTensor)
        x = data.to(device)
        y = label.to(device)
        mask = src_key_padding_mask(x).to(device)
        outputs = model(x,mask)
        # 손실 계산
        loss = criterion(outputs, y)
        
        # 역전파 및 가중치 업데이트
        optim.zero_grad()
        loss.backward()
        optim.step()
        
    if (i+1) % 10 == 0:
        print(f'epoch {(i+1)} :  Loss: {loss.item()}')

TypeError: Caught TypeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/myk/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/myk/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_2473082/1448273055.py", line 18, in forward
    out = self.classifier(out)
  File "/home/myk/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_2473082/1911602229.py", line 28, in forward
    out = out.reshape(self.batch, -1)
TypeError: reshape(): argument 'shape' (position 1) must be tuple of ints, but found element of type float at pos 0


In [ ]:


model.eval()

total =0
correct=0
incorrect_num_correct = np.zeros(120)
incorrect_num_answer = np.zeros(120)
with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y= label.to(device)
        mask = src_key_padding_mask(x)
        output = model.forward(x, mask)
        
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()
        
        for i in range(len(y)):
            if y[i]!=output_index[i]:
                k = int(y[i].cpu().numpy())
                incorrect_num_correct[k] += 1
                incorrect_num_answer[int(output_index[i].cpu().numpy())]+=1

    
    # 정확도 도출
    print("Accuracy of Test Data: {}%".format(100*correct/total))
   